# 栈与队列

- 栈：只能在表的顶端进出（访问受限）的线性表(后进先出)
- 队列：只能在表的前端删除，在表的后端插入（先进先出）的线性表

In [1]:
class stack:
    def __init__(self):
        self.items = []
    def push(self,item):
        self.items.append(item)
    def pop(self):
        if not self.is_empty():
            return self.items.pop()
    def is_empty(self):
        return len(self.items) == 0
    def top(self):
        if not self.is_empty():
            return self.items[-1]
    def size(self):
        return len(self.items)
class queue:
    def __init__(self):
        self.items = []
    def enqueue(self,item):
        self.items.append(item)
    def dequeue(self):
        if not self.is_empty():
            return self.items.pop(0)
    def is_empty(self):
        return len(self.items) == 0
    def front(self):
        if not self.is_empty():
            return self.items[0]
    def size(self):
        return len(self.items)

## 栈的基本应用
1. 进制转换

In [2]:
def convert_base(num, base):
    s = stack()
    if num == 0:
        return "0"
    characters = "0123456789ABCDEF"
    while(num > 0):
        remainder = num % base
        num = num // base
        s.push(characters[remainder])
    result = ""
    while(not s.is_empty()):
        result += s.pop()
    return result

print(convert_base(233, 16))  # Example: Convert 233 to hexadecimal
print(convert_base(233, 2))  # Example: Convert 233 to binary

E9
11101001


2. 括号匹配

In [3]:
def parentheses_match(s):
    p = stack()
    for char in s:
        if char in "({[":
            p.push(char)
        elif char in ")}]":
            if(p.is_empty()):
                return False
            else:
                top = p.pop()
                if(char == '(' and top == ')'
                   or char == '[' and top == ']'
                   or char == '{' and top == '}'):
                    break
    return p.is_empty()

print(parentheses_match("((a+b)*c)"))  # True
print(parentheses_match("((a+b)*c"))  # False

True
False


3. 栈混洗
- 入栈出栈可行操作序列分析 — 卡特兰数问题
条件：操作序列(2n位)的任意前m位，入栈数大于等于出栈数
- 计算卡特兰数
  - C(0) = 1
  - C(n) = Σ(C(i) * C(n-i-1)) for i in range(0, n)
    - C(n) = C(2n, n) / (n + 1)
- 计算括号匹配数
  - C(n) = C(2n, n) / (n + 1)

判断是否属于栈混洗：
- 维护一个栈，遍历入栈序列
- 对于每个元素，若栈顶元素等于出栈序列的当前元素，则出栈
- 否则，将当前元素入栈

In [4]:
def stackPermutation(B):
    S = stack()
    n = len(B)
    i = 1
    for k in range(n):
        while S.is_empty() or B[k] != S.top():
            if i > n:
                return False
            else:
                S.push(i)
                i += 1
        S.pop()
    return True

print(stackPermutation([1, 2, 3, 4]))  # True
print(stackPermutation([4, 3, 2, 1]))  # True

True
True


4. 表达式求值

- 分类：中缀，前缀，后缀表达式
- 前缀无需优先级和括号，易于求解
- 后缀（逆波兰）表达式也易于求值，三者可以相互转换

下面为后缀用栈的简单求值

In [5]:
def rpnevaluation(expr):
    s = stack()
    for char in expr:
        if char in "0123456789":
            s.push(int(char))
        elif char in "+-*/":
            b = s.pop()
            a = s.pop()
            if char == '+':
                s.push(a + b) 
            elif char == '-':
                s.push(a - b)
            elif char == '*':
                s.push(a * b)
            elif char == '/':
                s.push(a / b)
        elif char == '^':
            b = s.pop()
            a = s.pop()
            s.push(a ** b)
        elif char == '!':
            a = s.pop()
            fact = 1
            for i in range(1, a + 1):
                fact *= i
            s.push(fact)
    if s.is_empty():
        return None
    return s.pop()

#example usage
print(rpnevaluation("23*54*+"))
print(rpnevaluation("23+5*4-"))
print(rpnevaluation("23+5*4-6/"))
print(rpnevaluation("23+5*4-6/2^3!"))

26
21
3.5
6


中缀表达式需要先转换为rpn

In [6]:
def infix_to_postfix(expr):
    precedence = {'+':1, '-':1, '*':2, '/':2, '^':3, '!':4}
    output = []
    op_stack = stack()
    tokens = []
    i = 0
    
    # 分词
    while i < len(expr):
        if expr[i].isdigit():
            num = expr[i]
            while i+1 < len(expr) and expr[i+1].isdigit():
                i += 1
                num += expr[i]
            tokens.append(num)
        elif expr[i] in "+-*/^!()[]{}":
            tokens.append(expr[i])
        i += 1

    # 括号配对字典
    left_brackets = {'(': ')', '[': ']', '{': '}'}
    right_brackets = {')': '(', ']': '[', '}': '{'}

    for token in tokens:
        if token.isdigit():
            output.append(token)
        elif token in left_brackets:
            op_stack.push(token)
        elif token in right_brackets:
            while not op_stack.is_empty() and op_stack.top() != right_brackets[token]:
                output.append(op_stack.pop())
            op_stack.pop()  
        else:
            while (not op_stack.is_empty() and op_stack.top() not in left_brackets and
                   precedence.get(token, 0) <= precedence.get(op_stack.top(), 0)):
                output.append(op_stack.pop())
            op_stack.push(token)
        print(f"Token: {token}, Output: {output}, Stack: {op_stack.items}")
    while not op_stack.is_empty():
        output.append(op_stack.pop())
    return ''.join(output)

#exaple usage
infix_expr = "2*3+4-5/6*2+1"
postfix_expr = infix_to_postfix(infix_expr)
print("后缀表达式:", postfix_expr)
print("求值结果:", rpnevaluation(postfix_expr))

# example usage
infix_expr = "2+3*[4-5/(6^2)]+7!"
postfix_expr = infix_to_postfix(infix_expr)
print("后缀表达式:", postfix_expr)
print("求值结果:", rpnevaluation(postfix_expr))

Token: 2, Output: ['2'], Stack: []
Token: *, Output: ['2'], Stack: ['*']
Token: 3, Output: ['2', '3'], Stack: ['*']
Token: +, Output: ['2', '3', '*'], Stack: ['+']
Token: 4, Output: ['2', '3', '*', '4'], Stack: ['+']
Token: -, Output: ['2', '3', '*', '4', '+'], Stack: ['-']
Token: 5, Output: ['2', '3', '*', '4', '+', '5'], Stack: ['-']
Token: /, Output: ['2', '3', '*', '4', '+', '5'], Stack: ['-', '/']
Token: 6, Output: ['2', '3', '*', '4', '+', '5', '6'], Stack: ['-', '/']
Token: *, Output: ['2', '3', '*', '4', '+', '5', '6', '/'], Stack: ['-', '*']
Token: 2, Output: ['2', '3', '*', '4', '+', '5', '6', '/', '2'], Stack: ['-', '*']
Token: +, Output: ['2', '3', '*', '4', '+', '5', '6', '/', '2', '*', '-'], Stack: ['+']
Token: 1, Output: ['2', '3', '*', '4', '+', '5', '6', '/', '2', '*', '-', '1'], Stack: ['+']
后缀表达式: 23*4+56/2*-1+
求值结果: 9.333333333333334
Token: 2, Output: ['2'], Stack: []
Token: +, Output: ['2'], Stack: ['+']
Token: 3, Output: ['2', '3'], Stack: ['+']
Token: *, Output: 

In [7]:
def evaluate_infix(expr):
    precedence = {'+':1, '-':1, '*':2, '/':2, '^':3, '!':4}
    left_brackets = {'(': ')', '[': ']', '{': '}'}
    right_brackets = {')': '(', ']': '[', '}': '{'}
    op_stack = stack()
    val_stack = stack()
    tokens = []
    i = 0

    # 分词
    while i < len(expr):
        if expr[i].isdigit():
            num = expr[i]
            while i+1 < len(expr) and expr[i+1].isdigit():
                i += 1
                num += expr[i]
            tokens.append(num)
        elif expr[i] in "+-*/^!()[]{}":
            tokens.append(expr[i])
        i += 1

    def apply_op(op):
        if op == '!':
            a = val_stack.pop()
            fact = 1
            for j in range(1, a + 1):
                fact *= j
            val_stack.push(fact)
        else:
            b = val_stack.pop()
            a = val_stack.pop()
            if op == '+':
                val_stack.push(a + b)
            elif op == '-':
                val_stack.push(a - b)
            elif op == '*':
                val_stack.push(a * b)
            elif op == '/':
                val_stack.push(a / b)
            elif op == '^':
                val_stack.push(a ** b)

    for token in tokens:
        if token.isdigit():
            val_stack.push(int(token))
        elif token in left_brackets:
            op_stack.push(token)
        elif token in right_brackets:
            while not op_stack.is_empty() and op_stack.top() != right_brackets[token]:
                apply_op(op_stack.pop())
            op_stack.pop()
        else:
            while (not op_stack.is_empty() and op_stack.top() not in left_brackets and
                   precedence.get(token, 0) <= precedence.get(op_stack.top(), 0)):
                apply_op(op_stack.pop())
            op_stack.push(token)
    while not op_stack.is_empty():
        apply_op(op_stack.pop())
    return val_stack.pop()

print("中缀表达式直接求值:", evaluate_infix(infix_expr))

中缀表达式直接求值: 5053.583333333333


## 八皇后问题
- 回溯法

In [8]:
class Queen:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __eq__(self, other):
        return self.x == other.x or self.y == other.y or abs(self.x - other.x) == abs(self.y - other.y)
    
def solve_queens_stack(n):
    solutions = []
    s = stack()
    row = 0
    cols = [0] * n
    while True:
        found = False
        while cols[row] < n:
            new_queen = Queen(row, cols[row])
            if not any(new_queen == q for q in s.items):
                s.push(new_queen)
                cols[row] += 1
                row += 1
                found = True
                break
            cols[row] += 1
        if not found:
            if s.is_empty():
                break
            last_queen = s.pop()
            row = last_queen.x
            cols[row] = last_queen.y + 1
        if s.size() == n:
            solutions.append(list(s.items))
            last_queen = s.pop()
            row = last_queen.x
            cols[row] = last_queen.y + 1
    return solutions

def print_queens(queens):
    n = len(queens)
    for i in range(n):
        row = ['.'] * n
        row[queens[i].y] = 'Q'
        print(' '.join(row))
    print()

# 打印8皇后的所有解
for solution in solve_queens_stack(8):
    print_queens(solution)


## Queue的基本应用
1. 离散事件的模拟

In [9]:
import random
class Customer:
    def __init__(self, window, time):
        self.window = window
        self.time = time
    def __repr__(self):
        return f"Customer(window={self.window}, time={self.time})"

def best_window(windows):
    # 返回队列长度最短的窗口编号
    lengths = [w.size() for w in windows]
    return lengths.index(min(lengths))

def display_progress(windows, nWin, now):
    print(f"时间: {now}")
    for i in range(nWin):
        print(f"窗口{i}: {[c.time for c in windows[i].items]}",end=' ')
    print()
    print("-" * 30)

def simulate(nWin, servTime):
    windows = [queue() for _ in range(nWin)]
    for now in range(servTime):
        if random.randint(0, nWin) != 0:  # 新顾客以nWin/(nWin+1)概率到达
            c = Customer(window=None, time=random.randint(1, 98))
            c.window = best_window(windows)
            windows[c.window].enqueue(c)
        for i in range(nWin):
            if not windows[i].is_empty():
                windows[i].items[0].time -= 1
                if windows[i].items[0].time <= 0:
                    windows[i].dequeue()
        display_progress(windows, nWin, now)

# example usage
simulate(10, 10)

时间: 0
窗口0: [] 窗口1: [] 窗口2: [] 窗口3: [] 窗口4: [] 窗口5: [] 窗口6: [] 窗口7: [] 窗口8: [] 窗口9: [] 
------------------------------
时间: 1
窗口0: [12] 窗口1: [] 窗口2: [] 窗口3: [] 窗口4: [] 窗口5: [] 窗口6: [] 窗口7: [] 窗口8: [] 窗口9: [] 
------------------------------
时间: 2
窗口0: [11] 窗口1: [47] 窗口2: [] 窗口3: [] 窗口4: [] 窗口5: [] 窗口6: [] 窗口7: [] 窗口8: [] 窗口9: [] 
------------------------------
时间: 3
窗口0: [10] 窗口1: [46] 窗口2: [74] 窗口3: [] 窗口4: [] 窗口5: [] 窗口6: [] 窗口7: [] 窗口8: [] 窗口9: [] 
------------------------------
时间: 4
窗口0: [9] 窗口1: [45] 窗口2: [73] 窗口3: [28] 窗口4: [] 窗口5: [] 窗口6: [] 窗口7: [] 窗口8: [] 窗口9: [] 
------------------------------
时间: 5
窗口0: [8] 窗口1: [44] 窗口2: [72] 窗口3: [27] 窗口4: [79] 窗口5: [] 窗口6: [] 窗口7: [] 窗口8: [] 窗口9: [] 
------------------------------
时间: 6
窗口0: [7] 窗口1: [43] 窗口2: [71] 窗口3: [26] 窗口4: [78] 窗口5: [35] 窗口6: [] 窗口7: [] 窗口8: [] 窗口9: [] 
------------------------------
时间: 7
窗口0: [6] 窗口1: [42] 窗口2: [70] 窗口3: [25] 窗口4: [77] 窗口5: [34] 窗口6: [1] 窗口7: [] 窗口8: [] 窗口9: [] 
------------------------------
时间: 8